In [ ]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from scripts import solve_model as sm

In [ ]:
input_Mask = np.zeros(sm.N)
ablation_Mask = np.ones(sm.N, dtype = 'bool')

noise_mean = 0.
v_var = 3
s_var = 1e-5

R = np.eye(sm.N * 2)
R[0, 0] = v_var
R[1, 1] = v_var
R[2, 2] = v_var
R[3, 3] = s_var
R[4, 4] = s_var
R[5, 5] = s_var

In [ ]:
input_Mask[1] = 0.05

In [ ]:
result_dict = sm.run_Network(0, 20, 0.01, input_Mask, ablation_Mask = ablation_Mask, mode = 'standard')

In [ ]:
t_arr = result_dict['t']
vol_mat = result_dict['voltage_mat']
syn_mat = result_dict['syn_activity_mat']
nsteps = result_dict['steps']
V_th = result_dict['V_threshold']

In [ ]:
sm.Vth

In [ ]:
VsubVth = np.subtract(vol_mat, np.tile(V_th, (nsteps, 1))).transpose()
syn_mat_reshaped = syn_mat.transpose()

In [ ]:
plt.plot(t_arr[100:], VsubVth[0, 100:], linewidth = 2, color = 'blue'), 
plt.plot(t_arr[100:], VsubVth[1, 100:], linewidth = 2, color = 'red'),
plt.plot(t_arr[100:], VsubVth[2, 100:], linewidth = 2, color = 'orange')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')

blue_patch = mpatches.Patch(color='blue', label='Neuron #1')
red_patch = mpatches.Patch(color='red', label='Neuron #2')
orange_patch = mpatches.Patch(color='orange', label='Neuron #3')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[blue_patch, red_patch, orange_patch])
plt.savefig('voltage_dyn_true.png')

In [ ]:
plt.plot(t_arr[100:], syn_mat_reshaped[0, 100:], linewidth = 2, color = 'blue'),
plt.plot(t_arr[100:], syn_mat_reshaped[1, 100:], linewidth = 2, color = 'red'), 
plt.plot(t_arr[100:], syn_mat_reshaped[2, 100:], linewidth = 2, color = 'orange')
plt.xlabel('Time (s)')
plt.ylabel('Synaptic Activity')

blue_patch = mpatches.Patch(color='blue', label='Neuron #1')
red_patch = mpatches.Patch(color='red', label='Neuron #2')
orange_patch = mpatches.Patch(color='orange', label='Neuron #3')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[blue_patch, red_patch, orange_patch])
plt.savefig('syn_dyn_true.png')

In [ ]:
t_arr_c, combined_noised, Vth = sm.add_Noise(result_dict, noise_mean, v_var, s_var)
VsubVth_noised = np.subtract(combined_noised[:3, :], np.tile(V_th, (nsteps, 1)).transpose())

In [ ]:
plt.plot(t_arr_c[100:], VsubVth_noised[0, 100:], color = 'blue'),
plt.plot(t_arr_c[100:], VsubVth_noised[1, 100:], color = 'red'),
plt.plot(t_arr_c[100:], VsubVth_noised[2, 100:], color = 'orange')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')

blue_patch = mpatches.Patch(color='blue', label='Neuron #1')
red_patch = mpatches.Patch(color='red', label='Neuron #2')
orange_patch = mpatches.Patch(color='orange', label='Neuron #3')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[blue_patch, red_patch, orange_patch])
plt.savefig('voltage_dyn_noise.png')

In [ ]:
plt.plot(t_arr_c[100:], combined_noised[3, 100:], color = 'blue'),
plt.plot(t_arr_c[100:], combined_noised[4, 100:], color = 'red'),
plt.plot(t_arr_c[100:], combined_noised[5, 100:], color = 'orange')
plt.xlabel('Time (s)')
plt.ylabel('Synaptic Activity')

blue_patch = mpatches.Patch(color='blue', label='Neuron #1')
red_patch = mpatches.Patch(color='red', label='Neuron #2')
orange_patch = mpatches.Patch(color='orange', label='Neuron #3')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[blue_patch, red_patch, orange_patch])
plt.savefig('syn_dyn_noise.png')

In [ ]:
result_dict_ref = sm.run_Network(0, 20, 0.01, input_Mask, ablation_Mask = ablation_Mask, mode='model')

In [ ]:
t_arr_ref = result_dict_ref['t']
vol_mat_ref = result_dict_ref['voltage_mat']
syn_mat_ref = result_dict_ref['syn_activity_mat']
nsteps_ref = result_dict_ref['steps']
V_th_ref = result_dict_ref['V_threshold']

In [ ]:
sm.Vth

In [ ]:
VsubVth_ref = np.subtract(vol_mat_ref, np.tile(V_th_ref, (nsteps_ref, 1))).transpose()
syn_mat_reshaped_ref = syn_mat_ref.transpose()

In [ ]:
x_init_true = np.hstack([vol_mat[100, :], syn_mat[100, :]])
x_init = sm.compute_Init(x_init_true, [30, 30, 30, 1e-4, 1e-4, 1e-4])

P_init = np.eye(len(x_init))
P_init[0, 0] = 30
P_init[1, 1] = 30
P_init[2, 2] = 30
P_init[3, 3] = 1e-4
P_init[4, 4] = 1e-4
P_init[5, 5] = 1e-4

y = combined_noised[:, 100:]

In [ ]:
np.dot(P_init, np.linalg.inv(np.add(P_init, R)))

In [ ]:
xf_list, Pf_list, xu_list, Pu_list = sm.run_EFK(x_init, P_init, 0, 0.01, R, y)

In [ ]:
xu_mat = np.hstack(xu_list)
v_est_mat = xu_mat[:3, :]
s_est_mat = xu_mat[3:, :]
Vth_mat = np.tile(V_th, (len(xu_list), 1)).transpose()

v_est_final = np.subtract(v_est_mat, Vth_mat)

In [ ]:
plt.plot(t_arr[100:], v_est_final[0, :], linewidth = 2, color = 'black'),
plt.plot(t_arr[100:], VsubVth[0, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], VsubVth_ref[0, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('V1_compare')

In [ ]:
plt.plot(t_arr[100:], v_est_final[1, :], linewidth = 2, color = 'black'), 
plt.plot(t_arr[100:], VsubVth[1, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], VsubVth_ref[1, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('V2_compare')

In [ ]:
plt.plot(t_arr[100:], v_est_final[2, :], linewidth = 2, color = 'black'), 
plt.plot(t_arr[100:], VsubVth[2, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], VsubVth_ref[2, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('V3_compare')

In [ ]:
plt.plot(t_arr[100:], s_est_mat[0, :], linewidth = 2, color = 'black'), 
plt.plot(t_arr[100:], syn_mat_reshaped[0, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], syn_mat_reshaped_ref[0, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Synaptic Activity')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('S1_compare')

In [ ]:
plt.plot(t_arr[100:], s_est_mat[1, :], linewidth = 2, color = 'black'), 
plt.plot(t_arr[100:], syn_mat_reshaped[1, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], syn_mat_reshaped_ref[1, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Synaptic Activity')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('S2_compare')

In [ ]:
plt.plot(t_arr[100:], s_est_mat[2, :], linewidth = 2, color = 'black'), 
plt.plot(t_arr[100:], syn_mat_reshaped[2, 100:], linewidth = 2, color = 'orange'),
plt.plot(t_arr[100:], syn_mat_reshaped_ref[2, 100:], linewidth = 2, color = 'blue')
plt.xlabel('Time (s)')
plt.ylabel('Synaptic Activity')

black_patch = mpatches.Patch(color='black', label='Estimated')
orange_patch = mpatches.Patch(color='orange', label='True')
red_patch = mpatches.Patch(color='blue', label='Assumed Model')
plt.legend(bbox_to_anchor=(1.05, 1), loc=5, borderaxespad=0., handles=[black_patch, orange_patch, red_patch])
plt.savefig('S3_compare')

In [ ]:
MSE_0 = sm.compute_MSE(VsubVth_noised[:, 110:], VsubVth[:, 110:])
MSE_1 = sm.compute_MSE(v_est_final[:, 10:], VsubVth[:, 110:])
MSE_0, MSE_1

In [ ]:
v_error = np.subtract(v_est_final, VsubVth[:, 100:])
s_error = np.subtract(s_est_mat, syn_mat_reshaped[:, 100:])

plt.plot(t_arr[200:], v_error[0, 100:])
plt.xlabel('Time (s)')
plt.ylabel('Error')
plt.savefig('Error_plot_S2')